In [1]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px

from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

import re

import spacy
from spacy.lang.fr.stop_words import STOP_WORDS

import nltk ## traitement sur le texte
nltk.download('punkt') ## nltk tokenizer requiert punkt package
from nltk import word_tokenize ## fonction pour tockenizer
from nltk.corpus import stopwords ##  stopwords dans différentes langues
nltk.download('stopwords')## nltk corpus requiert punkt package
nltk.download('wordnet') ## nltk stem requiert punkt package
from nltk.stem import WordNetLemmatizer # Lemmatizer

from tqdm.auto import tqdm # for progress bar

from vaderSentiment_fr.vaderSentiment import SentimentIntensityAnalyzer
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import time

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\theoj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\theoj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\theoj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv("csv/scraped_reviews.csv", sep=";")
df.head()

,consumer_name,consumer_nb_review_writed,stars,title_review,content_review,date_experience,company,category
0,Ghestin K,1,5.0,Les meilleurs,"Large catalogue, SAV a l’écoute et compréhensi...",15 mars 2021,www.lepetitvapoteur.com,food_beverages_tobacco
1,Sylvain,2,5.0,Un large choix de produits,"Un large choix de produits, du Matos de qualit...",15 mars 2021,www.lepetitvapoteur.com,food_beverages_tobacco
2,Helrat,4,5.0,Toujours parfait,"Toujours parfait. Depuis 4 ans, je commande to...",14 mars 2021,www.lepetitvapoteur.com,food_beverages_tobacco
3,Laura Perrault,1,5.0,Site aussi bon que d'habitude ou l'on…,Site aussi bon que d'habitude ou l'on trouve p...,8 mars 2021,www.lepetitvapoteur.com,food_beverages_tobacco
4,HENRY KARINE,2,5.0,Super!,"Présentation agréable, conseils rapides et ada...",11 mars 2021,www.lepetitvapoteur.com,food_beverages_tobacco


# drop na comment

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141006 entries, 0 to 141005
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   consumer_name              141000 non-null  object 
 1   consumer_nb_review_writed  141006 non-null  int64  
 2   stars                      141000 non-null  float64
 3   title_review               141000 non-null  object 
 4   content_review             125918 non-null  object 
 5   date_experience            29406 non-null   object 
 6   company                    141006 non-null  object 
 7   category                   141006 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 8.6+ MB


In [4]:
df = df[df["content_review"].notna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125918 entries, 0 to 141005
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   consumer_name              125912 non-null  object 
 1   consumer_nb_review_writed  125918 non-null  int64  
 2   stars                      125918 non-null  float64
 3   title_review               125918 non-null  object 
 4   content_review             125918 non-null  object 
 5   date_experience            25494 non-null   object 
 6   company                    125918 non-null  object 
 7   category                   125918 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 8.6+ MB


# join title and content

In [5]:
# df["review"] = df["title_review"] + " " + df["content_review"]
# df.head()

In [6]:
# df = df.drop(["consumer_name", "consumer_nb_review_writed", "date_experience", "company", "category"], axis=1)
# df.head()

In [7]:
df["review"] = df["content_review"]

# PREPROCESS

In [8]:
def clean_text(txt: str) -> str:
    """
    clean_text : remove "numbers, spec caracs, <br \>" in string
    
    params:
        txt -> string : string to clean
    return:
        txt -> string : cleaned string
    """

    # en regex "^" spécifie de ne garder que les carac spécifiés
    #print(re.sub(r"[A-Z|a-z| |']", r" ", txt))
    #txt = re.sub(r"[^A-Z|a-z]", r" ", txt)
    carac = chr(8217)
    txt = re.sub(r"['|,|.|" + carac + r"]", r" ", txt)
    txt = txt.lower()
    
    return txt

In [9]:
def remove_stop_words(txt: list) -> list:
    """
    remove_stop_words : remove useless words in string
    
    params:
        txt -> list : list of string to remove stop words
    return:
        txt -> string : list of string without stop words
    """

    txt = [x for x in txt if x not in stop_words]
    return txt

In [10]:
lemmatizer = WordNetLemmatizer()

def lemmatize(tokens: list) -> list:
    """
    Convert words from list to their common base forms
    Params
        tokens -> list : list of words to lemmatize
    Return
        lemmatized_tokens -> list: list of lemmatized words
    """
    
    for i in ["a", "v"]:
        tokens = list(map(lambda x: lemmatizer.lemmatize(x, i), tokens))
    
    return tokens

In [11]:
def preprocess(txt: str) -> list:
    """
    Cleans, Tokenize, lemmatize text and remove stop words
    Params:
        txt -> str: text to preprocess
    Return:
        txt -> str: preprocessed text
    """
    
    txt = clean_text(txt)
    
    txt = word_tokenize(txt)
    
    txt = remove_stop_words(txt)
    
    txt = lemmatize(txt)
    
    txt = " ".join(txt)
    
    return txt

In [19]:
print(stop_words)

{'j’', 'rendre', 'doivent', 'surtout', 'mille', 'peux', 'semblaient', 'se', 'serait', 'tres', 'neuvième', 'une', 'troisième', 'fais', 'ouvert', 'dedans', 'suivre', 'different', 'je', "l'", 'ouverts', 'delà', 'm’', 'etc', 'abord', "s'", 'étaient', 'malgre', 'parle', 'aucun', 'troisièmement', 'laquelle', 'moi', 'compris', 'façon', 'suivants', 'oust', 'quelconque', 'excepté', "n'", 'prealable', 'quel', 'egalement', 'autrement', 'seize', 'dans', 'douzième', "d'", 'lors', 'derriere', 'six', 'deja', 'quand', 'hou', 'eu', 'etais', 'enfin', 'etant', 'soi-même', 'un', 'cinquante', "t'", 'quanta', 'unes', 'moindres', 'celui-ci', 'tellement', 'soi', 'toutes', 'memes', 'meme', 'partant', 'assez', 'treize', 'trois', 'duquel', 'des', 'devra', 'en', 'dit', 'vers', 'celle-ci', 'sur', 'donc', 'qui', 'relativement', 'sans', 'auraient', 'été', 'onze', 'suffit', 'différente', 'revoici', 'tant', 'avons', 'suivant', 'diverse', 'nos', 'suffisant', 'restent', 'sait', 'dès', 'tente', 'toi', 'quelles', 'dix-sep

In [12]:
tqdm.pandas()

stop_words = set(STOP_WORDS)

df["preprocess_review"] = df["review"].progress_apply(preprocess)

In [13]:
print(df["review"][0])
print(df["preprocess_review"][0])

Large catalogue, SAV a l’écoute et compréhensif, site facile d’utilisation avec de nombreux guides, livraison simple et rapide, nombreuses réductions et cadeaux.De loin le meilleur site de cigarette électronique sur le marché français.À quand une application?
large catalogue sav l écoute compréhensif site facile d utilisation guide livraison simple rapide réductions cadeaux loin meilleur site cigarette électronique marché français application ?


# sentiment analyse

In [32]:
def sentiment_analyse(column):

    senti_list = []
    senti_nb = []
    senti_confidence = []
    
    for i in df[column]:
        vs = tb(i).sentiment
        senti_nb.append(vs[0])
        senti_confidence.append(vs[1])
        if (vs[0] > 0):
            senti_list.append(2)
        elif (vs[0] < 0):
            senti_list.append(0)
        else:
            senti_list.append(1) 

    df["sentiment"] = senti_list
    df["sentiment_nb"] = senti_nb
    df["sentiment_confidence"] = senti_confidence
    return df


#     senti_list = []
#     senti_confidence = []
#     SIA = SentimentIntensityAnalyzer()
    
#     for i in df[column]:
#         #score = SIA.polarity_scores_max(i)
#         score = SIA.polarity_scores(i)
#         if score["compound"] > 0.05:
#             senti_confidence.append(score["pos"])
#             senti_list.append(2)
#         elif score["compound"] >= -0.05 and score["compound"] <= 0.05:
#             senti_confidence.append(score["neu"])
#             senti_list.append(1)
#         else:
#             senti_confidence.append(score["neg"])
#             senti_list.append(0)
            
#     df["sentiment"] = senti_list
#     df["sentiment_confidence"] = senti_confidence
#     return df


#df = sentiment_analyse("review")
df = sentiment_analyse("preprocess_review")
df.head()

,consumer_name,consumer_nb_review_writed,stars,title_review,content_review,date_experience,company,category,review,preprocess_review,sentiment,sentiment_confidence,sentiment_nb
0,Ghestin K,1,5.0,Les meilleurs,"Large catalogue, SAV a l’écoute et compréhensi...",15 mars 2021,www.lepetitvapoteur.com,food_beverages_tobacco,"Large catalogue, SAV a l’écoute et compréhensi...",large catalogue sav l écoute compréhensif site...,2,0.266667,0.266667
1,Sylvain,2,5.0,Un large choix de produits,"Un large choix de produits, du Matos de qualit...",15 mars 2021,www.lepetitvapoteur.com,food_beverages_tobacco,"Un large choix de produits, du Matos de qualit...",large choix produits matos qualité livraison r...,2,0.200000,0.300000
2,Helrat,4,5.0,Toujours parfait,"Toujours parfait. Depuis 4 ans, je commande to...",14 mars 2021,www.lepetitvapoteur.com,food_beverages_tobacco,"Toujours parfait. Depuis 4 ans, je commande to...",parfait 4 ans commande matériel diy liquide e-...,2,0.533333,0.211667
3,Laura Perrault,1,5.0,Site aussi bon que d'habitude ou l'on…,Site aussi bon que d'habitude ou l'on trouve p...,8 mars 2021,www.lepetitvapoteur.com,food_beverages_tobacco,Site aussi bon que d'habitude ou l'on trouve p...,site bon d habitude l trouve pleins petits con...,2,0.350000,0.207500
4,HENRY KARINE,2,5.0,Super!,"Présentation agréable, conseils rapides et ada...",11 mars 2021,www.lepetitvapoteur.com,food_beverages_tobacco,"Présentation agréable, conseils rapides et ada...",présentation agréable conseils rapides adaptés...,2,0.400000,0.450000


In [33]:
print(df["sentiment"].value_counts())
print(df["stars"].value_counts())

2    112696
1      7511
0      5711
Name: sentiment, dtype: int64
5.0    108835
4.0     11983
3.0      2084
1.0      2070
2.0       946
Name: stars, dtype: int64


In [36]:
df[df["sentiment"]==0]

,consumer_name,consumer_nb_review_writed,stars,title_review,content_review,date_experience,company,category,review,preprocess_review,sentiment,sentiment_confidence,sentiment_nb
34,Isabelle L-v,1,5.0,Client depuis 2 ans société et conseil…,Client depuis 2 ans société et conseil au top ...,23 février 2021,www.lepetitvapoteur.com,food_beverages_tobacco,Client depuis 2 ans société et conseil au top ...,client 2 ans société conseil top recommande ye...,0,0.125000,-0.040000
132,Dominique Jouan,5,5.0,Rien à dire,"Rien à dire, tout est parfait, sauf que je n'a...",NaN,www.lepetitvapoteur.com,food_beverages_tobacco,"Rien à dire, tout est parfait, sauf que je n'a...",rien parfait n reçu masque lpv croyais temps pis,0,0.800000,-0.400000
168,Catherine Françoise Fragu,1,5.0,Nickel,Nickel Rapidité et qualité Équipe très réactiv...,NaN,www.lepetitvapoteur.com,food_beverages_tobacco,Nickel Rapidité et qualité Équipe très réactiv...,nickel rapidité qualité équipe très réactive j...,0,0.262500,-0.042500
237,Client,2,1.0,Première expérience très mauvaise,Première expérience très très mauvaise...J'ai ...,NaN,www.lepetitvapoteur.com,food_beverages_tobacco,Première expérience très très mauvaise...J'ai ...,expérience très très mauvaise j passé commande...,0,0.271429,-0.258571
271,Wilfrid,1,2.0,Tout d'abord nous étions très satisfait…,Tout d'abord nous étions très satisfait de la ...,2 février 2021,www.comptoirdesvignes.fr,food_beverages_tobacco,Tout d'abord nous étions très satisfait de la ...,d étions très satisfait boutique y trouver reç...,0,0.422222,-0.087778
...,...,...,...,...,...,...,...,...,...,...,...,...,...
140775,Matthias Carrere,1,5.0,Au top rien à dire meilleur site de…,Au top rien à dire meilleur site de vente de p...,2 mars 2021,www.tonnycat.com,vehicles_transportation,Au top rien à dire meilleur site de vente de p...,top rien meilleur site vente pièces détachées,0,0.300000,-0.182500
140784,GEORGES,1,5.0,J'ai acheté 2 pièces pour un rzr,J'ai acheté 2 pièces pour un rzr; toit et pare...,28 février 2021,www.tonnycat.com,vehicles_transportation,J'ai acheté 2 pièces pour un rzr; toit et pare...,j acheté 2 pièces rzr ; toit pare brise dernie...,0,0.262500,-0.022500
140825,Mr DONOVAN Rosati,2,1.0,Bonjour je ne suis pas satisfait car…,Bonjour je ne suis pas satisfait car j'ai eu u...,22 février 2021,www.tonnycat.com,vehicles_transportation,Bonjour je ne suis pas satisfait car j'ai eu u...,bonjour satisfait j erreurs commande coup j pe...,0,0.466667,-0.126302
140874,CEDRIC veysauto,1,4.0,Dommage que personne m est recontacte…,Dommage que personne m est recontacte comme an...,11 février 2021,www.tonnycat.com,vehicles_transportation,Dommage que personne m est recontacte comme an...,dommage m recontacte annoncé,0,0.500000,-0.300000


In [35]:
df_for_train = df[(df["sentiment_confidence"] > 0.8)]
#df[(df["sentiment_confidence"] > 0.7) & (df["stars"] == 1)]
display(df_for_train.head())

print(df_for_train["sentiment"].value_counts())

,consumer_name,consumer_nb_review_writed,stars,title_review,content_review,date_experience,company,category,review,preprocess_review,sentiment,sentiment_confidence,sentiment_nb
21,Sébastien GILLET,1,5.0,Toujours très content de ce site,"Toujours très content de ce site, des produits...",26 février 2021,www.lepetitvapoteur.com,food_beverages_tobacco,"Toujours très content de ce site, des produits...",très content site produits proposés rapidité l...,2,1.00,1.00
22,christine lemle,3,5.0,Super site,Super site . Jamais déçue .,26 février 2021,www.lepetitvapoteur.com,food_beverages_tobacco,Super site . Jamais déçue .,super site jamais déçue,2,0.95,0.60
117,saint yves CECILE,1,5.0,Parfait,"Parfait, comme d'habitude 48h c'est génial",NaN,www.lepetitvapoteur.com,food_beverages_tobacco,"Parfait, comme d'habitude 48h c'est génial",parfait d habitude 48h c génial,2,0.90,0.85
143,Patricia,2,5.0,Super produits à tout les prix et…,Super produits à tout les prix et surtout très...,NaN,www.lepetitvapoteur.com,food_beverages_tobacco,Super produits à tout les prix et surtout très...,super produits prix très rapide niveaux livrai...,2,0.85,0.70
153,Jo. Nj,2,5.0,Très bon service,"Très bon service, jamais un seul soucis",NaN,www.lepetitvapoteur.com,food_beverages_tobacco,"Très bon service, jamais un seul soucis",très bon service jamais soucis,2,1.00,1.00


2    8869
0     200
1      18
Name: sentiment, dtype: int64


In [37]:
df_for_train[df_for_train["sentiment"]==0]

,consumer_name,consumer_nb_review_writed,stars,title_review,content_review,date_experience,company,category,review,preprocess_review,sentiment,sentiment_confidence,sentiment_nb
1443,Denis LOUIS,1,5.0,Très satisfait que la livraison…,Très satisfait que la livraison corresponde ex...,NaN,au-droit-de-bouchon.com,food_beverages_tobacco,Très satisfait que la livraison corresponde ex...,très satisfait livraison corresponde commande ...,0,1.000000,-0.200000
1919,Mazloum Nour,2,5.0,Incroyable,J’avais vraiment 0 mal de crâne en lendemain d...,NaN,alcoool.fr,food_beverages_tobacco,J’avais vraiment 0 mal de crâne en lendemain d...,j vraiment 0 mal crâne lendemain soirée faut t...,0,1.000000,-1.000000
3106,Marie Robin,1,5.0,Toujours très satisfaite des produits…,Toujours très satisfaite des produits et servi...,NaN,www.oliquide.com,food_beverages_tobacco,Toujours très satisfaite des produits et servi...,très satisfaite produits service sav top réact...,0,1.000000,-0.125000
3231,Marie-Odile BAZIN,1,1.0,Je suis un peu déçu.Je n'ai pas reçu ma…,Je suis un peu déçu.Je n'ai pas reçu ma comman...,NaN,nutri-naturel.com,food_beverages_tobacco,Je suis un peu déçu.Je n'ai pas reçu ma comman...,déçu n reçu commande devait arriver 29/12/2020...,0,1.000000,-1.000000
3314,linda mine,1,1.0,déçu de la composition du produit…,déçu de la composition du produit contrairemen...,NaN,nutri-naturel.com,food_beverages_tobacco,déçu de la composition du produit contrairemen...,déçu composition produit contrairement j lu,0,1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
136261,Lionel FLEURET,3,4.0,bonjour,bonjour les roulements commandés ont ete monte...,16 mars 2021,www.123roulement.com,vehicles_transportation,bonjour les roulements commandés ont ete monte...,bonjour roulements commandés ete monter rien c...,0,1.000000,-0.350000
137029,aubry tony,1,5.0,impec,Rien à dire! impeccable,4 mars 2021,www.lacameraembarquee.fr,vehicles_transportation,Rien à dire! impeccable,rien ! impeccable,0,1.000000,-0.350000
138617,Marie Jo Basle,2,5.0,Jamai déçue.,"Jamaisi déçue.Servcie, disponibilité, accueil ...",NaN,izipark-nantes.fr,vehicles_transportation,"Jamaisi déçue.Servcie, disponibilité, accueil ...",jamaisi déçue servcie disponibilité accueil top,0,1.000000,-1.000000
139343,Oriol,2,5.0,Parfait,Troisième expérience de location chez Ford et ...,NaN,www.fordrent.fr,vehicles_transportation,Troisième expérience de location chez Ford et ...,expérience location ford déçu l accueil presta...,0,0.933333,-0.316667


In [ ]:
df_for_train.to_csv("df_for_train.csv", sep=";", index=False)